In [13]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import urllib.request
import re
import cv2
import matplotlib.pyplot as plt
import os 

In [15]:
def clean_links(links: list): 
    new = []
    for i in range(len(links)): 
        # check the link validation
        if "m'{sid:,cturl:" in  links[i] or "m{&quot;sid&quot;:&quot;&quot;" in  links[i] or "https://static.ah.nl/" in links[i]:
            # if true skip this step
            pass
        else: 
            # else append image link to new array
            new.append(links[i])
    # remove duplicate 
    return np.unique(new)

# def resize(): 

def download_image(image_name :  str ):
    # https://www.bing.com/images/search?q=dog&FORM=HDRSC3
    array_links = []
    for i in range(100):
        try: 
            url = requests.get(f'https://www.bing.com/images/search?q={image_name}&form=HDRSC3&first={i}')
            soup = BeautifulSoup(url.content, 'lxml')
            #  find the image tag 
            get_image = soup.find_all(name = "div", attrs={"class" : "imgpt"})
            # if succed to get the image tag 
            if get_image: 
                # print(get_image)
                for attributes in str(get_image).split("src"):
                    for i in attributes.split(" "): 
                        if "https://tse2.mm.bin" in i: 
                            url = i
                            url = url.replace(" ", "")
                            array_links.append(url.replace("=", "").replace('"', '').replace(" ", ""))
        except Exception as e: 
            print(f'error : {e} or image tag not detected')
    # get the all link 
    array_links = clean_links(array_links)
    # start to download
    for i in range(len(array_links)):
        # check the image path, if the path is not exist, create the path 
        if os.path.exists(f'./{image_name}'):
            print(f'donwloading : {array_links[i]}')
            # full path for saving the image
            full_path = f'./{image_name}/{image_name}{i}.jpeg'
            urllib.request.urlretrieve(array_links[i], full_path)
        else:
            #  if the path is exist
            os.mkdir(f"./{image_name}")
            print(f'donwloading : {array_links[i]}')
            full_path = f'./{image_name}/{image_name}{i}.jpeg'
            urllib.request.urlretrieve(array_links[i], full_path)
    
    print("succesfully get " + str(len(array_links)) + " images")   
     




In [16]:
# url_image
# example, if you want to scrape rendang image 
array_links = download_image(image_name="rendang")
array_links

donwloading : https://tse2.mm.bing.net/th/id/ODF.QkgEBNS94uQ9D2jkAQCn8w?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/ODF.Ylowxy59eLlDg0nheoqOqg?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/ODF.rLuBO0gkR1dKUTLXHrzGOg?w12&amp;h12&amp;c7&amp;rs1&amp;p0&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/OIP.-tsh23luSQfoKV7ckEqX9wHaE8?w197&amp;h180&amp;c7&amp;r0&amp;o5&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/OIP.-tsh23luSQfoKV7ckEqX9wHaE8?w214&amp;h180&amp;c7&amp;r0&amp;o5&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/OIP.-tsh23luSQfoKV7ckEqX9wHaE8?w225&amp;h180&amp;c7&amp;r0&amp;o5&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/OIP.1SX7RS1PwCh6G6snqm5F8gAAAA?w155&amp;h180&amp;c7&amp;r0&amp;o5&amp;pid1.7
donwloading : https://tse2.mm.bing.net/th/id/OIP.1SX7RS1PwCh6G6snqm5F8gAAAA?w157&amp;h180&amp;c7&amp;r0&amp;o5&amp;pid1.7
donwloading : https://tse2.mm.bing.net/

ValueError: unknown url type: 'vlees/3,murl:https://static.ah.nl/static/recepten/img_083461_1024x748_JPG.jpg,turl:https://tse2.mm.bing.net/th?idOIP.MGlvurWKGq3EThb8oFoEZAHaFa&amp;pid15.1,md5:30696fbab58a1aadc44e16fca05a0464,shkey:T1U1/Fa7UpMCRzaAv0h46rj322uh8U83fdUMvx/TdP8,t:Recepten'